# Determine Word to PCA Associations

This notebook is designed to run PCA over the document embeddings and calculate words-pca associations and document centroid-pca associations for each principal component. 

In [1]:
from pathlib import Path
import os
import re

from gensim.models import Word2Vec
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import plotnine as p9
from PIL import ImageColor
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook
import wordcloud

In [2]:
journal_map_df = pd.read_csv("../exploratory_data_analysis/output/biorxiv_article_metadata.tsv", sep="\t")
journal_map_df.head()

,author_type,heading,category,document,doi
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735
1,regular article,new results,systems biology,775270_v1.xml,10.1101/775270
2,regular article,new results,genetics,242404_v1.xml,10.1101/242404
3,regular article,new results,neuroscience,872994_v1.xml,10.1101/2019.12.11.872994
4,regular article,new results,developmental biology,080853_v2.xml,10.1101/080853


# Get the Word Vectors

Save the word vectors to disk, so later sections have easy access.

In [3]:
model = Word2Vec.load("../word_vector_experiment/output/word2vec_models/300/biorxiv_300.model")

In [4]:
word_vector_map = {
    word:model.wv[word]
    for word in tqdm_notebook(model.wv.vocab.keys())
}

In [5]:
word_vector_df = pd.DataFrame.from_dict(word_vector_map, orient='index')

word_vector_df.to_csv(
    "output/word_pca_similarity/word_vectors_300.tsv.xz", 
    sep="\t", compression="xz"
)

word_vector_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
AbstractAdaptation,-0.285844,-0.080816,0.131058,-0.164448,0.213790,0.289946,0.198592,-0.613607,-0.233766,0.169285,...,-0.005080,-0.127008,-0.108292,0.227025,0.219927,0.020532,-0.016353,-0.153878,0.047264,0.304174
response,1.982980,-0.921032,0.322336,-1.819926,1.794423,-2.737348,-1.291250,-1.636407,1.895059,-1.916617,...,2.644842,0.406725,1.360475,-2.210262,-3.219634,-1.696714,0.730268,-1.190313,-0.815162,-1.846863
selection,-0.991861,-1.071227,-0.182926,-4.167827,2.136675,2.197686,-0.744608,-0.422629,2.331867,-1.475493,...,1.926673,1.632876,-2.516936,0.276007,-3.518850,-2.685090,0.832754,4.678514,-0.146672,-2.214616
polygenic,0.410014,-1.328640,0.992850,-1.669407,1.523946,-0.893682,0.450540,-3.377987,1.740303,3.022100,...,-3.074039,-2.214306,-1.203037,-0.051227,-2.269419,1.449286,0.433501,4.297900,1.160446,-0.868293
phenotypes,-0.161185,-0.631166,2.041816,-1.221121,0.177153,0.605349,1.185401,-1.300211,0.637753,-0.737132,...,2.640480,3.109912,-0.838520,0.948513,-3.318489,-2.088322,2.560162,-0.631192,1.679850,0.125102


# PCA the Documents

Run PCA over the documents. Generates 50 principal components, but can generate more or less.

In [3]:
n_components = 50
random_state = 100

In [5]:
biorxiv_articles_df = pd.read_csv(
    Path("..")/
    Path("word_vector_experiment")/
    Path("output/word2vec_output")/
    Path("biorxiv_all_articles_300.tsv.xz"), 
    sep="\t"
)

In [6]:
reducer = PCA(
    n_components = n_components,
    random_state = random_state
)

embedding = reducer.fit_transform(
    biorxiv_articles_df[[f"feat_{idx}" for idx in range(300)]].values
)

pca_df = (
    pd.DataFrame(embedding, columns=[f"pca{dim}" for dim in range(1, n_components+1, 1)])
    .assign(document=biorxiv_articles_df.document.values.tolist())
    .merge(journal_map_df[["category", "document", "doi"]], on="document")
)

In [8]:
(
    pd.DataFrame(
        reducer.components_,
        columns=[f"{dim+1}" for dim in range(reducer.components_.shape[1])]
    )
    .to_csv(
        "output/word_pca_similarity/pca_components.tsv", 
        sep="\t", index=False
    )
)

# Calculate Word-PCA Cosine Similarity

Once PCA has finished, there are now 50 different principal components. The association between every word and principal component is calculated via [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) (cosine of the angle between two vectors).

In [6]:
word_vector_df = pd.read_csv(
    "output/word_pca_similarity/word_vectors_300.tsv.xz",
    sep="\t", index_col=0
)

In [7]:
# 1 - cosine distance = cosine similarity
word_pca_similarity = (
    1 - cdist(
        word_vector_df.values, 
        reducer.components_, 
        'cosine'
    )
)

word_pca_similarity.shape

(1158806, 50)

In [8]:
word_pca_sim_df = (
    pd.DataFrame(word_pca_similarity, columns=[f"pca{dim}_cossim" for dim in range(1, n_components+1, 1)])
    .assign(word=word_vector_df.index.tolist())
)

# for files greater than a 1GB
if n_components > 40:
    word_pca_sim_df.to_csv(
        f"output/word_pca_similarity/word_pca_cos_sim_{n_components}_pcs.tsv.xz", 
        sep="\t", index=False, compression="xz"
    )

else:
    word_pca_sim_df.to_csv(
        f"output/word_pca_similarity/word_pca_cos_sim_{n_components}_pcs.tsv", 
        sep="\t", index=False
    )

word_pca_sim_df.head()

,pca1_cossim,pca2_cossim,pca3_cossim,pca4_cossim,pca5_cossim,pca6_cossim,pca7_cossim,pca8_cossim,pca9_cossim,pca10_cossim,...,pca42_cossim,pca43_cossim,pca44_cossim,pca45_cossim,pca46_cossim,pca47_cossim,pca48_cossim,pca49_cossim,pca50_cossim,word
0,-0.045667,-0.078511,0.066551,-0.173033,0.080864,0.083595,-0.141889,-0.066019,0.059657,0.000733,...,-0.080560,0.012606,-0.129806,0.121300,-0.117969,-0.047289,0.023706,0.032255,-0.047201,AbstractAdaptation
1,0.062452,-0.376775,0.249009,0.231773,0.160104,-0.118593,0.213980,-0.095030,0.161049,-0.275582,...,0.087217,-0.054528,-0.057721,-0.064015,0.077887,-0.027870,-0.017259,0.000280,-0.060391,response
2,0.376841,0.238236,0.070698,-0.052009,0.227408,-0.160790,0.160619,0.025369,0.261476,-0.055970,...,0.012074,-0.080928,-0.097589,0.058252,0.043420,0.065997,0.019776,0.068211,-0.190315,selection
3,0.343602,0.201131,0.242376,0.067431,0.033333,-0.131094,0.184101,-0.040821,-0.025277,0.153308,...,-0.002069,-0.036506,-0.001910,0.036544,0.124395,0.054872,0.107793,0.058838,-0.048196,polygenic
4,0.087592,0.210725,0.402009,0.210496,0.117443,-0.103775,0.188247,0.121864,0.003405,0.012910,...,0.043147,-0.018047,-0.020037,-0.079398,0.087133,0.051841,0.024561,0.009387,-0.040963,phenotypes


# Generate Word Clouds for the PC dimensions

Given word to principal component association, next step is to generate word clouds for each principal component. The word clouds have orange representing words that are most similar to the principal component and blue as words most dissimilar to the principal component.

In [9]:
class PolarityColorFunc():
    def __init__(
        self, word_class_map, 
        positive_key="positive", positive="#ef8a62",
        negative_key="negative", negative="#67a9cf", 
        default="#f7f7f7"
    ):
        self.positive_class = positive
        self.negative_class = negative
        self.positive_key = positive_key
        self.negative_key = negative_key
        self.default_class = default
        
        self.words_to_color_map = word_class_map
    
    def get_color_mapper(self, word):
        if word in self.words_to_color_map[self.positive_key]:
            return ImageColor.getrgb(self.positive_class)
        #wordcloud.get_single_color_func(self.positive_class)
        elif word in self.words_to_color_map[self.negative_key]:
            return ImageColor.getrgb(self.negative_class)
        else:
            return ImageColor.getrgb(self.default_class)
    
    def __call__(self, word, **kwargs):
        return self.get_color_mapper(word)

In [10]:
pca_dimensions = [f"pca{dim}_cossim" for dim in range(1, n_components+1, 1)]
for pc, component in tqdm_notebook(enumerate(pca_dimensions, start=1)):
    word_class_map = {}
    
    word_class_map['negative'] = (
        word_pca_sim_df
        .sort_values(component, ascending=True)
        .head(100)
        .assign(**{component:lambda x: x[component].abs().values.tolist()})
        .assign(**{component:lambda x: x[component]/x[component].max()})
        [["word", component]]
        .to_dict(orient='records')
    )

    word_class_map['positive'] = (
        word_pca_sim_df
        .sort_values(component, ascending=False)
        .assign(**{component:lambda x: x[component]/x[component].max()})
        .head(100)
        [["word", component]]
        .to_dict(orient='records')
    )
    
    polarity_color_map = PolarityColorFunc({
        word_class:set(map(lambda x: x['word'], word_class_map[word_class]))
        for word_class in word_class_map
    })
    
    pc = f"{pc}" if pc > 9 else f"0{pc}"
    
    polarity_cloud = (
        wordcloud
        .WordCloud(background_color="white", width=1024, height=768, collocations=False)
        .generate_from_frequencies({
            record['word']:record[component]
            for word_class in word_class_map
            for record in word_class_map[word_class]
        })
        .recolor(color_func=polarity_color_map)
        .to_file(f"output/word_pca_similarity/figures/pca_{pc}_cossim_word_cloud.png")
    )

# Document Centroid Cosine Similarity

Finally this section calculates document centroid to principal component associations. This means the higher score the higher the association is between a document category and given principal component.

In [11]:
document_centroid_df = (
    journal_map_df[["document", "category"]]
    .merge(biorxiv_articles_df, on="document")
    .groupby("category")
    .agg({
        f"feat_{dim}":"mean"
        for dim in range(300)
    })
    .reset_index()
)
document_centroid_df.head()

,category,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_290,feat_291,feat_292,feat_293,feat_294,feat_295,feat_296,feat_297,feat_298,feat_299
0,animal behavior and cognition,-0.385377,-0.092016,-0.203736,0.263906,0.245076,-0.078879,-0.211472,-0.109519,0.327646,...,0.025290,0.158810,0.224406,-0.095334,-0.741804,-0.363689,0.426329,0.593084,-0.476876,-0.248656
1,biochemistry,-0.324586,-0.149939,-0.634199,-0.013107,0.047569,-0.209220,-0.363617,0.135296,0.395399,...,0.358709,0.030319,0.255905,-0.115047,-0.389403,-0.129026,0.340299,0.360118,-0.325085,0.385691
2,bioengineering,-0.177445,-0.218462,-0.547813,0.409557,0.096975,-0.271289,-0.257142,0.124036,0.459858,...,0.288691,0.016546,0.211963,-0.189489,-0.438316,-0.191111,0.190613,0.307242,-0.461027,0.074699
3,bioinformatics,-0.365937,-0.082641,-0.353911,0.112348,0.084613,-0.240221,-0.211348,0.099509,0.395627,...,0.043020,0.119018,0.034140,0.167894,-0.599870,-0.139959,0.564357,0.604689,-0.189464,0.060606
4,biophysics,-0.468284,-0.244065,-0.705333,0.367350,0.280165,-0.247054,-0.545020,0.020306,0.452312,...,0.227000,-0.042084,0.350163,0.062992,-0.381906,-0.205850,0.278241,0.512302,-0.388975,0.081560


In [12]:
# 1 - cosine distance = cosine similarity
centroid_pca_similarity = (
    1 - cdist(
        document_centroid_df[[f"feat_{dim}" for dim in range(300)]].values, 
        reducer.components_, 
        'cosine'
    )
)

centroid_pca_similarity.shape

(29, 50)

In [13]:
centroid_pca_df = (
    pd.DataFrame(centroid_pca_similarity, columns=[f"pca{dim}_cossim" for dim in range(1, n_components+1, 1)])
    .assign(category=document_centroid_df.category.tolist())
    [["category"] + [f"pca{dim}_cossim" for dim in range(1, n_components+1, 1)]]
)
centroid_pca_df.to_csv("output/word_pca_similarity/centroid_pca_cos_sim.tsv", sep="\t", index=False)
centroid_pca_df.head()

,category,pca1_cossim,pca2_cossim,pca3_cossim,pca4_cossim,pca5_cossim,pca6_cossim,pca7_cossim,pca8_cossim,pca9_cossim,...,pca41_cossim,pca42_cossim,pca43_cossim,pca44_cossim,pca45_cossim,pca46_cossim,pca47_cossim,pca48_cossim,pca49_cossim,pca50_cossim
0,animal behavior and cognition,0.645319,-0.170817,0.086692,0.083208,0.118532,-0.192764,0.180107,0.027869,0.129040,...,0.152150,-0.005014,-0.116218,-0.086474,0.013780,0.121861,-0.003086,0.044292,0.058777,-0.049905
1,biochemistry,0.009803,0.132332,-0.295027,0.162962,0.189058,-0.218619,0.294752,-0.168105,-0.010645,...,0.173266,-0.000167,-0.134454,-0.125951,0.007188,0.162472,0.008644,0.063255,0.059423,-0.073842
2,bioengineering,0.334599,-0.085237,-0.227295,0.098525,-0.009232,-0.155151,0.356780,0.109760,0.051121,...,0.186219,0.014620,-0.133430,-0.121224,0.022696,0.156565,0.007188,0.049414,0.075145,-0.075060
3,bioinformatics,0.621906,0.311122,-0.155286,0.246931,-0.007207,-0.149435,0.256250,0.049583,0.041310,...,0.131572,0.000110,-0.099808,-0.092103,0.016044,0.115652,0.006172,0.037317,0.052331,-0.054182
4,biophysics,0.334841,-0.081345,-0.353738,0.167893,0.213872,-0.258759,0.314320,0.006609,-0.031146,...,0.154543,0.009418,-0.114683,-0.115968,0.008532,0.136191,0.009831,0.049935,0.070473,-0.058445
